# pandas - Data Wrangling: Join, Combine, and Reshape

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

## Hierarchical Indexing

Enables us to have multiple index levels on an axis. It may looks abstract, but it provides a way for us to work with higher dimensional data in a lower dimensional form.

In [3]:
data = Series(np.random.randn(9), 
             index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                    [1,2,3,1,3,1,2,2,3]])
data

In [4]:
data.index # MultiIndex

In [5]:
data['b']

In [6]:
data['b':'c']

In [7]:
data.loc[['b', 'd']]

In [8]:
data.loc[:, 2]

In [9]:
data.unstack() # reforming the data into a DataFrame

In [10]:
data.unstack().stack() # inverse

In [11]:
frame = DataFrame(np.arange(12).reshape((4, 3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

In [12]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

In [13]:
frame['Ohio']

### Reordering and Sorting Levels

In [14]:
frame

In [15]:
frame.swaplevel('key1', 'key2')

In [16]:
frame.sort_index(level=1) # sort inner index

In [17]:
frame.swaplevel(0, 1).sort_index(level=0) # switch key1(0) with key2(1) value then sort the outer value

### Summary Statistics by Level

In [18]:
frame.sum(level='key2') # sums the value for each col in key2

In [19]:
frame.sum(level='color', axis=1) # Ohio has Green and Red, Colorado only has Green (this sums according to each col)

### Indexing with a DataFrame’s columns

In [20]:
frame = DataFrame({'a': range(7), 'b': range(7, 0, -1),
                   'c': ['one', 'one', 'one', 'two', 'two','two', 'two'],
                   'd':[0,1,2,0,1,2,3]})
frame

In [21]:
frame2 = frame.set_index(['c', 'd'])
frame2

In [22]:
frame.set_index(['c', 'd'], drop=False) # including the dropped columns

In [23]:
frame.reset_index() # opposite of set_index, moves hierarchical index level into columns

## Combining and Merging Datasets

### Database-Style DataFrame Joins

In [24]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df1

In [25]:
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})
df2

In [26]:
pd.merge(df1, df2) # join operations

In [27]:
pd.merge(df1, df2, on='key') # specify on join operations is helpful

In [28]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df3

In [29]:
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})
df4

In [30]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey') # if column names are different in each object, you can specify them seperately

In [31]:
pd.merge(df1, df2, how='outer') # outer join (taking all columns from df1 and df2, if one does not exist then result to NA)

In [32]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})
df1

In [33]:
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})
df2

In [34]:
pd.merge(df1, df2, on='key', how='left') # left join (using all key combinations found in the left table)

In [35]:
pd.merge(df1, df2, how='inner') # inner join (use key combinations observed in both tables)

In [36]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
left

In [37]:
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
right

In [40]:
pd.merge(left, right, on=['key1', 'key2'], how='outer') # outer join, specifying on multiple keys

In [41]:
pd.merge(left, right, on='key1')

In [42]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right')) # adding suffixes

### Merging on Index

In [43]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                   'value': range(6)})
left1

In [44]:
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

In [47]:
pd.merge(left1, right1, left_on='key', right_index=True)

In [48]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

#### Dealing with hierarchically index data

In [49]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                   'key2': [2000, 2001, 2002, 2001, 2002],
                   'data': np.arange(5.)})
lefth

In [50]:
righth = DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'], [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])
righth

In [51]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True) # having to indicate multiple columns to merge on as a list

In [52]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer') # merging and handling duplicate index values with outer join

In [53]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                  index=['a', 'c', 'e'],
                  columns=['Ohio', 'Nevada'])
left2

In [54]:
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'],
                   columns=['Missouri', 'Alabama'])
right2

In [55]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True) # using indexes of both sides

In [56]:
left2.join(right2, how='outer') # left outer join

In [57]:
left1.join(right1, on='key')

In [58]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'],
                    columns=['New York', 'Oregon'])
another

In [59]:
left2.join([right2, another]) # merging multiple DF

In [60]:
left2.join([right2, another], how='outer') # outer join on multiple DF

### Concatenating Along an Axis

In [61]:
arr = np.arange(12).reshape((3, 4))
arr

In [62]:
np.concatenate([arr, arr], axis=1)

In [69]:
s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])
s1, s2, s3

In [67]:
pd.concat([s1, s2, s3]) # concat all Series

In [70]:
pd.concat([s1, s2, s3], axis=1) # concating across columns

In [71]:
s4 = pd.concat([s1, s3])
s4

In [73]:
pd.concat([s1, s3], axis=1)

In [74]:
pd.concat([s1, s3], join='inner')

In [78]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

In [79]:
result.unstack()

In [80]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

In [81]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                columns=['one', 'two'])
df1

In [82]:
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                columns=['three', 'four'])
df2

In [83]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

In [84]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

In [86]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

In [87]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1

In [88]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

In [89]:
pd.concat([df1, df2], ignore_index=True)

### Combining Data with Overlap

There is another data combination situation that can’t be expressed as either a merge or concatenation operation. *You may have two datasets whose indexes overlap in full or part.*

In [90]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
a

In [91]:
b = Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])
b

In [92]:
b[-1] = np.nan
b

In [93]:
np.where(pd.isnull(b), b, a)

In [99]:
b[:-2].combine_first(a[2:])

In [100]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})
df1

In [101]:
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})
df2

In [102]:
df1.combine_first(df2) # patching missing datas

## Reshaping and Pivoting

In [103]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'],
                 name='number'))
data

In [104]:
result = data.stack()
result

In [105]:
result.unstack()

In [106]:
result.unstack(0)

In [107]:
result.unstack('state') # same as above

In [109]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])

In [110]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

In [111]:
data2.unstack() # if there are values that aren't found in a level then value is set to NA

In [112]:
data2.unstack().stack()

In [113]:
data2.unstack().stack(dropna=False)

In [114]:
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))
df

In [115]:
df.unstack('state')

In [116]:
df.unstack('state').stack('side')

### Pivoting “Long” to “Wide” Format

A common way to store multiple time series in databases and CSV is in so-called long or stacked format.

In [134]:
data = pd.read_csv('../input/pandas-data-loading-storage-file-formats/macrodata.csv')
data.head()

In [135]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

In [136]:
data = data.reindex(columns=columns) # drop any other columns other than above
data

In [137]:
data.index = periods.to_timestamp('D', 'end')

In [138]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

This is the so-called long format for multiple time series or other observational data with two or more keys. Each row in the table represents a single observation.

In [139]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

In [142]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

In [143]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

In [144]:
pivoted['value'][:5]

In [145]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

### Pivoting “Wide” to “Long” Format

In [146]:
df = DataFrame({'key': ['foo', 'bar', 'baz'],
                'A': [1, 2, 3],
                'B': [4, 5, 6],
                'C': [7, 8, 9]})
df

In [147]:
melted = pd.melt(df, ['key']) # merges multiple columns into 1 (inverse of pivot)
melted

In [148]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

In [149]:
reshaped.reset_index()

In [150]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B']) # using group identifiers

In [151]:
pd.melt(df, value_vars=['A', 'B', 'C']) # without group identifiers